In [1]:
import os

candidates = ["/VOL_D/teresa.w/Dataset_Xception/full_dataset",
              "/teresa.w/Dataset_Xception/full_dataset",]

for p in candidates:
    print(p, "->",os.path.isdir(p))

/VOL_D/teresa.w/Dataset_Xception/full_dataset -> True
/teresa.w/Dataset_Xception/full_dataset -> False


In [2]:
import sys
print(sys.version)
print(sys.executable)

3.10.9 (main, Mar  1 2023, 18:23:06) [GCC 11.2.0]
/opt/anaconda3/bin/python3


In [3]:
import sys

python_exec = sys.executable
print("Using python:", python_exec)

Using python: /opt/anaconda3/bin/python3


In [4]:
!{python_exec} -m pip install --user --upgrade pip

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


In [5]:
!{python_exec} -m pip install --user "tensorflow<2.16" matplotlib scikit-learn

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


In [11]:
import os
import tensorflow as tf
from tensorflow.keras.applications import Xception
from tensorflow.keras.applications.xception import preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.models import Model

In [12]:
BASE_DIR = "/VOL_D/teresa.w/Dataset_Xception/full_dataset"

In [13]:
TRAIN_DIR = os.path.join(BASE_DIR, "train")
VAL_DIR = os.path.join(BASE_DIR, "validation")
TEST_DIR = os.path.join(BASE_DIR, "test")

print("Train:", TRAIN_DIR, "->", os.path.isdir(TRAIN_DIR))
print("Val  :", VAL_DIR, "->", os.path.isdir(VAL_DIR))
print("Test :", TEST_DIR, "->", os.path.isdir(TEST_DIR))

Train: /VOL_D/teresa.w/Dataset_Xception/full_dataset/train -> True
Val  : /VOL_D/teresa.w/Dataset_Xception/full_dataset/validation -> True
Test : /VOL_D/teresa.w/Dataset_Xception/full_dataset/test -> True


In [16]:
img_size = (299, 299)
batch_size = 32

train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
)

val_datagen = ImageDataGenerator(
    preprocessing_function = preprocess_input

)

train_gen = train_datagen.flow_from_directory(
    TRAIN_DIR,
    target_size=img_size,
    batch_size=batch_size,
    class_mode="categorical"

)

val_gen = val_datagen.flow_from_directory(
    VAL_DIR,
    target_size=img_size,
    batch_size=batch_size,
    class_mode="categorical"

)

num_classes = train_gen.num_classes
print("Classes:", train_gen.class_indices)

Found 4211 images belonging to 3 classes.
Found 1403 images belonging to 3 classes.
Classes: {'0': 0, '1': 1, '2': 2}


In [18]:
base_model = Xception(
    weights="imagenet",
    include_top=False,
    input_shape=(299, 299, 3)
)

base_model.trainable = False

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation="relu")(x)
x = Dropout(0.5)(x)
outputs = Dense(num_classes, activation = "softmax")(x)

model = Model(inputs= base_model.input, outputs=outputs)

model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_4 (InputLayer)        [(None, 299, 299, 3)]        0         []                            
                                                                                                  
 block1_conv1 (Conv2D)       (None, 149, 149, 32)         864       ['input_4[0][0]']             
                                                                                                  
 block1_conv1_bn (BatchNorm  (None, 149, 149, 32)         128       ['block1_conv1[0][0]']        
 alization)                                                                                       
                                                                                                  
 block1_conv1_act (Activati  (None, 149, 149, 32)         0         ['block1_conv1_bn[0][0]'] 

In [20]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-3),
    loss="categorical_crossentropy",
    metrics=["accuracy"]

)

historyl = model.fit(
    train_gen,
    epochs = 5,
    validation_data=val_gen
)

Epoch 1/5


2025-12-01 15:10:24.001138: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8907
2025-12-01 15:10:24.043080: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2025-12-01 15:10:26.455607: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2025-12-01 15:10:44.116941: I external/local_xla/xla/service/service.cc:168] XLA service 0x7f21f1784840 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2025-12-01 15:10:44.116964: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA H100 80GB HBM3, Compute Capability 9.0
2025-12-01 15:10:44.121227: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1764601844.194560 3197890 device_compiler.h:

132/132 [==============================] - 79s 375ms/step - loss: 0.2114 - accuracy: 0.9145 - val_loss: 0.1479 - val_accuracy: 0.9387
Epoch 2/5
132/132 [==============================] - 49s 374ms/step - loss: 0.1470 - accuracy: 0.9456 - val_loss: 0.1381 - val_accuracy: 0.9537
Epoch 3/5
132/132 [==============================] - 49s 375ms/step - loss: 0.1155 - accuracy: 0.9542 - val_loss: 0.1231 - val_accuracy: 0.9522
Epoch 4/5
132/132 [==============================] - 49s 370ms/step - loss: 0.0987 - accuracy: 0.9649 - val_loss: 0.1347 - val_accuracy: 0.9579
Epoch 5/5
132/132 [==============================] - 49s 371ms/step - loss: 0.0954 - accuracy: 0.9611 - val_loss: 0.1394 - val_accuracy: 0.9579


In [21]:
import os

save_dir = "/VOL_D/asim.n/xception_xray_project"
os.makedirs(save_dir, exist_ok=True)

model_path = os.path.join(save_dir, "xception_full_dataset.h5") 
model.save(model_path)
print("Model saved to:",model_path)

/home/asim.n/.local/lib/python3.10/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Model saved to: /VOL_D/asim.n/xception_xray_project/xception_full_dataset.h5
